In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
# ls

In [26]:
# cd drive/MyDrive

In [27]:
import os
import cv2
from PIL import Image
import tensorflow as tf
from keras import backend as K
from keras.models import load_model
from tensorflow.keras.utils import img_to_array
# from tensorflow.keras.optimizers import Adam, RMSprop
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
imagegen = ImageDataGenerator()
import matplotlib.pyplot as plt

In [28]:
train = imagegen.flow_from_directory("Marine_Dataset/train/", class_mode="categorical", shuffle=True, batch_size=32, target_size=(50, 50))
test = imagegen.flow_from_directory("Marine_Dataset/test/", class_mode="categorical", shuffle=True, batch_size=32, target_size=(50, 50))
val = imagegen.flow_from_directory("Marine_Dataset/val/", class_mode="categorical", shuffle=True, batch_size=32, target_size=(50, 50))

Found 4642 images belonging to 4 classes.
Found 1037 images belonging to 4 classes.
Found 115 images belonging to 4 classes.


In [29]:
pre_trained_model = tf.keras.applications.EfficientNetV2S(input_shape=(50, 50, 3), include_top=False, weights="imagenet")

In [30]:
for layer in pre_trained_model.layers:
  print(layer.name)
  layer.trainable = False

input_2
rescaling_1
stem_conv
stem_bn
stem_activation
block1a_project_conv
block1a_project_bn
block1a_project_activation
block1a_add
block1b_project_conv
block1b_project_bn
block1b_project_activation
block1b_drop
block1b_add
block2a_expand_conv
block2a_expand_bn
block2a_expand_activation
block2a_project_conv
block2a_project_bn
block2b_expand_conv
block2b_expand_bn
block2b_expand_activation
block2b_project_conv
block2b_project_bn
block2b_drop
block2b_add
block2c_expand_conv
block2c_expand_bn
block2c_expand_activation
block2c_project_conv
block2c_project_bn
block2c_drop
block2c_add
block2d_expand_conv
block2d_expand_bn
block2d_expand_activation
block2d_project_conv
block2d_project_bn
block2d_drop
block2d_add
block3a_expand_conv
block3a_expand_bn
block3a_expand_activation
block3a_project_conv
block3a_project_bn
block3b_expand_conv
block3b_expand_bn
block3b_expand_activation
block3b_project_conv
block3b_project_bn
block3b_drop
block3b_add
block3c_expand_conv
block3c_expand_bn
block3c_expan

In [31]:
last_layer = pre_trained_model.get_layer('block6o_add')
last_output = last_layer.output
x = tf.keras.layers.GlobalMaxPooling2D()(last_output)
x = tf.keras.layers.Flatten()(last_output)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(4, activation='sigmoid')(x)

In [32]:
model = tf.keras.Model(pre_trained_model.input, x)

In [33]:
model.compile(optimizer='adam', loss=tf.keras.losses.categorical_crossentropy, metrics=['accuracy'], run_eagerly=True)

In [34]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 50, 50, 3)]  0           []                               
                                                                                                  
 rescaling_1 (Rescaling)        (None, 50, 50, 3)    0           ['input_2[0][0]']                
                                                                                                  
 stem_conv (Conv2D)             (None, 25, 25, 24)   648         ['rescaling_1[0][0]']            
                                                                                                  
 stem_bn (BatchNormalization)   (None, 25, 25, 24)   96          ['stem_conv[0][0]']              
                                                                                            

In [35]:
enet_classifier = model.fit(train, steps_per_epoch=(2520//32), epochs = 16, validation_data=val, validation_steps=(33//32), batch_size = 32, verbose = 1)

Epoch 1/16
78/78 [==============================] - 45s 558ms/step - loss: 2.2534 - accuracy: 0.7210 - val_loss: 0.8476 - val_accuracy: 0.7188
Epoch 2/16
78/78 [==============================] - 43s 553ms/step - loss: 0.5703 - accuracy: 0.7956 - val_loss: 0.2970 - val_accuracy: 0.9062
Epoch 3/16
78/78 [==============================] - 40s 516ms/step - loss: 0.4357 - accuracy: 0.8398 - val_loss: 0.4522 - val_accuracy: 0.8438
Epoch 4/16
78/78 [==============================] - 41s 519ms/step - loss: 0.4461 - accuracy: 0.8301 - val_loss: 0.2527 - val_accuracy: 0.8750
Epoch 5/16
78/78 [==============================] - 41s 521ms/step - loss: 0.3975 - accuracy: 0.8413 - val_loss: 0.4294 - val_accuracy: 0.8125
Epoch 6/16
78/78 [==============================] - 43s 548ms/step - loss: 0.4116 - accuracy: 0.8530 - val_loss: 0.4658 - val_accuracy: 0.8125
Epoch 7/16
78/78 [==============================] - 43s 548ms/step - loss: 0.3569 - accuracy: 0.8577 - val_loss: 0.4866 - val_accuracy: 0.9062

In [36]:
result = model.evaluate(test, batch_size=32)
print("[Loss, Accuracy] = ", result)

33/33 [==============================] - 8s 255ms/step - loss: 0.3307 - accuracy: 0.8795
[Loss, Accuracy] =  [0.3307231664657593, 0.8794599771499634]


In [37]:
import numpy as np

In [38]:
test_img = 'test_image.png'

In [39]:
test = cv2.imread(test_img)
test = cv2.resize(test, (50, 50))
test = np.array(test)
test = np.expand_dims(test, axis=0)
print("Shape of Validation Data: ", test.shape)
pred = model.predict(test)

Shape of Validation Data:  (1, 50, 50, 3)
1/1 [==============================] - 0s 197ms/step


In [40]:
def pred_img(index):
    if index == 0:
        return 'Biodegradable'
    elif index == 1:
        return 'Non-Biodegradable'
    elif index == 3:
        return 'Reduce/Reuse/Recycle-able'
    else:
        return 'Others'


op = []
pred = np.array(pred.tolist())[0]
index = pred.tolist().index(max(pred))
# print(index)
print("Input Test Image falls under the Category:", pred_img(index))
print("Accuracy = ", max(pred))

Input Test Image falls under the Category: Non-Biodegradable
Accuracy =  0.9884325861930847
